## Prepare MCC MAP batch file

In [1]:
import pandas as pd
from pandas import DataFrame
from os.path import join
import re
downloads = r'c:\Users\balob\Documents\GITLAB\RB_BD\DATA'
dmi_allowed_list = 'DMI.dmi_allowed_list_export_Sat_Jan_11_2020_after.csv'
dmi_netpfx = 'DMI.dmi_netpfx_export_Thu_Jan_09_2020.csv'

In [17]:
# Подготовить DataFrame на основе заданного Allowed List

# Задать название MCC MAP профайла
profile = 'DIAL_MAP'

# Задать список Allowed Lists, которые будут использоваться для генерации MCC MAP профайла
s1_al_name = 'Partner_DIAL_COM'
s2_al_name = 'P4_DIAL_COMB'
s4_al_name = 'MB_DIAL_COMB'
s5_al_name = ''

# Задать имена используемых [sts] Sponsor Profile
s1_sts_profile_name = 'Partner_DIALOQ'
s2_sts_profile_name = 'P4_DIALOQ'
s4_sts_profile_name = 'MB_DIALOQ'
s5_sts_profile_name = ''

sponsor_list = [s1_al_name, s2_al_name, s4_al_name, s5_al_name]

# Prepare allowed list

#Like expression for Allowed List
df_al=pd.read_csv(join(downloads, dmi_allowed_list), usecols=['Sponsor','PLMN code'])
df_al['SPONSOR'] = df_al['Sponsor'].replace(to_replace={r'^[Pp][Aa][rtner]?[\s\w-]*|^S1_[\s\w-]*':s1_sts_profile_name,
                                                        r'^[Pp]4[\s\w-]*|^S2_[\s\w-]*':s2_sts_profile_name,
                                                        r'^[Mm][Bb][\s\w-]*|^S4_[\s\w-]*':s4_sts_profile_name,
                                                        r'^SMART[\s\w-]*|^S5_[\s\w-]*':s5_sts_profile_name}, regex=True)

df_al.rename({'PLMN code':'TADIG'}, axis=1, inplace=True)

df_al = df_al.loc[df_al.Sponsor.isin(sponsor_list),['SPONSOR','TADIG']]

# Prepare mccmnc list from netpfx

df_np=pd.read_csv(join(downloads, dmi_netpfx), usecols=['Operator','MCC']).drop_duplicates()
df_np.rename({'Operator':'TADIG'}, axis=1, inplace=True)

df_mcc_map = pd.merge(df_al, df_np, how='left', on='TADIG')

# Missing MCC

print('Missing MCC:')
display(df_mcc_map[df_mcc_map.MCC.isnull()])

# Duplicated MCC for TADIG

print('Duplicated MCC for TADIG:')
duplicated_mcc_list = [x[0] for x in
                       df_mcc_map.groupby('TADIG',as_index=False)['SPONSOR'].
                       count().values if x[1] > 1]
display(df_mcc_map[df_mcc_map.TADIG.isin(duplicated_mcc_list)])

# Prepare DataFrame with SPONSOR and MCC only

df_mcc = df_mcc_map[['SPONSOR','MCC']].drop_duplicates()

df_mcc['COUNTRY_NAME'] = '@'
df_mcc['Profile'] = profile

#*******************************************************************
# Adjustments if needed

# df_mcc.loc[df_mcc.MCC.isin(['520']),'SPONSOR'] = 'S4_COMB_STS'

#*******************************************************************

# Check for Duplicated MCC

print('Duplicated MCC for TADIG:')
duplicated_mcc_list = [x[0] for x in
                       df_mcc.groupby(['SPONSOR','MCC'], as_index=False)['Profile'].
                       count().values if x[1] > 1]
display(df_mcc[df_mcc.MCC.isin(duplicated_mcc_list)])

# The result

print('\nMCC MAP list:')
display(df_mcc.head(3))
display(df_mcc.groupby('SPONSOR')['MCC'].count())

Missing MCC:


,SPONSOR,TADIG,MCC
16,Partner_DIALOQ,IRLH3,NaN


Duplicated MCC for TADIG:


,SPONSOR,TADIG,MCC
122,Partner_DIALOQ,JAMDC,363.0
123,Partner_DIALOQ,JAMDC,338.0


Duplicated MCC for TADIG:


,SPONSOR,MCC,COUNTRY_NAME,Profile



MCC MAP list:


,SPONSOR,MCC,COUNTRY_NAME,Profile
0,Partner_DIALOQ,714.0,@,DIAL_MAP
1,MB_DIALOQ,502.0,@,DIAL_MAP
2,Partner_DIALOQ,401.0,@,DIAL_MAP


SPONSOR
MB_DIALOQ          7
P4_DIALOQ         24
Partner_DIALOQ    90
Name: MCC, dtype: int64

In [3]:
# Create TOC file to CREATE MCC MAP list

result_file_name = f'{profile.lower()}_create.rcjson'

ouf=open(join(downloads, result_file_name),'w')
#ouf.write('Some text\n')
#ouf.write(str(25))

num_val = df_mcc.count()[0]-1

t1='dmi_mcc_map.mcc_map.create,COUNTRY="",MCC="'
t2='",PROFILE_REF="'
t3='",SPONSOR_REF="'
t4='";\n'

for i, [sponsor, country, mcc, profile] in enumerate(df_mcc[['SPONSOR','COUNTRY_NAME','MCC','Profile']].values):
    ouf.write(t1+str(mcc)+t2+profile+t3+sponsor+t4)

ouf.close()

In [19]:
# Create Batch file to CREATE MCC MAP list

result_file_name = 'mcc_map_create.rcjson'

ouf=open(join(downloads, result_file_name),'w')
#ouf.write('Some text\n')
#ouf.write(str(25))

num_val = df_mcc.count()[0]-1

t1='{"params":{"country":"'
t2='","mcc":"'
t3='","profile":"'
t4='","sponsorReference":"'
t5='"},"type":"create","caption":"dmi_mcc_map_mcc_map","objectId":"DMI MCC MAP.dmi_mcc_map_mcc_map","serviceId":"DMI MCC MAP"}'

ouf.write('[')
for i, [sponsor, country, mcc, profile] in enumerate(df_mcc[['SPONSOR','COUNTRY_NAME','MCC','Profile']].values):
    ouf.write(t1+country+t2+str(mcc)+t3+profile+t4+sponsor+t5)
    if i < num_val:
        ouf.write(',')
ouf.write(']')

ouf.close()

In [3]:
# Create Batch file to DELETE MCC MAP list

result_file_name = 'mcc_map_delete.rcjson'

file_mcc_map_dmi = 'DMI MCC MAP.dmi_mcc_map_mcc_map_export_Sat_Jan_11_2020.csv'

df_mcc_delete = pd.read_csv(join(downloads, file_mcc_map_dmi))
df_mcc_delete.rename({'Sponsor reference':'SPONSOR','Country':'COUNTRY_NAME'}, axis=1, inplace=True)
df_mcc_delete.COUNTRY_NAME.fillna('', inplace=True)

ouf=open(join(downloads, result_file_name), 'w')

num_val = df_mcc_delete.count()[0]-1

t1='{"params":{"country":"'
t2='","mcc":"'
t3='","profile":"'
t4='","sponsorReference":"'
t5='"},"type":"remove","caption":"dmi_mcc_map_mcc_map","objectId":"DMI MCC MAP.dmi_mcc_map_mcc_map","serviceId":"DMI MCC MAP"}'

ouf.write('[')
for i, [sponsor, country, mcc, profile] in enumerate(df_mcc_delete[['SPONSOR','COUNTRY_NAME','MCC','Profile']].values):
    ouf.write(t1+str(country)+t2+str(mcc)+t3+profile+t4+sponsor+t5)
    if i < num_val:
        ouf.write(',')
ouf.write(']')

ouf.close()